In [1]:
import pytraj as pt
import numpy as np
import pandas as pd
import tqdm
import sys
import gc
import copy
import nglview as nv

In [6]:
strucDir='structures/'
viewPDBname='Piezo_PIP2.prot_only.pdb'
trajName='Piezo_PIP2.prot_only.dcd'
topName='Piezo_PIP2.prot_only.parm7'
viewStruc=pt.load(strucDir+'Piezo_PIP2.prot_only.pdb')
iterTraj=pt.iterload(strucDir+trajName,
                     top=strucDir+topName)

print viewStruc
print iterTraj

pytraj.Trajectory, 1 frames: 
Size: 0.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           
pytraj.TrajectoryIterator, 1592 frames: 
Size: 2.000000 (GB)
<Topology: 70296 atoms, 4254 residues, 12 mols, PBC with box type = ortho>
           


In [24]:
color_scheme=[
    ['blue','ARG LYS HIS HIP HID HIE HSD'],
    ['red','ASP GLU'],
    ['green','SER THR ASN GLN'],
    ['white','CYS SEC GLY PRO ALA VAL ILE LEU MET PHE TYR TRP']
]

In [68]:
view=nv.show_pytraj(viewStruc)
view.clear_representations()
view.add_representation('cartoon',color='white',alpha=.5)
view.add_representation('licorice',
                        color='blue',
                        selection='HSD LYS ARG')
view

NGLWidget()

In [80]:
from pytraj.utils.progress import ProgressBarTrajectory

In [ ]:
basicResidueIDs=[res.original_resid for res in \
                 viewStruc.top.residues if res.name in ['HSD','LYS','ARG']]
basicResidueSASAtable=pd.DataFrame(np.arange(iterTraj.n_frames),columns=['Frame'])
basicResidueSASAtable['System']='Piezo_PIP2'
commandList=['surf :%g'%iterTraj.top.residue(resID-1).original_resid \
             for resID in basicResidueIDs]
pbar=ProgressBarTrajectory(iterTraj,style='tqdm')
SASA_data=pt.compute(commandList,pbar)
for ii,resID in tqdm.tqdm_notebook(enumerate(basicResidueIDs)):
    basicResidueSASAtable[
            '%s_%g'%(iterTraj.top.residue(resID-1).name,resID)
        ]=SASA_data[ii]
basicResidueSASAtable.to_csv('basicResidueSASA.csv',index=False)
basicResidueSASAtable.head()